# LLM Finetune - Mistral 7B

This studio will let you fine-tune a Mistral 7B model, and request an inference through the API. 

To start, make sure you **switch to a A10G machine**.

In [1]:
import torch
assert torch.cuda.is_available(), "Please switch to a GPU instance (1xA10G recommended)"

AssertionError: Please switch to a GPU instance (1xA10G recommended)

### Prepare the dataset

First off, download and prepare the dataset. The following defaults to the Alpaca dataset (`--dataset alpaca`), but you can fine-tune on the Dolly dataset (`--dataset dolly`), or bring your own CSV (`--dataset csv`).

In the latter case, you need to provide a CSV file with the following 3 columns
```
instruction input output
```
and pass it as the `--csv_path <data.csv>` argument to the script.

In [ ]:
!python llm-finetune/prepare_dataset.py --dataset alpaca

### Fine-tune the base Mistral 7B model on the dataset

You can now fine-tune your model on the data. This script will automatically run across all available GPUs.

In [ ]:
!python llm-finetune/finetune.py --dataset alpaca

### Try generating text

You can now generate text using your fine-tuned model. Feel free to adjust the prompt as needed.

In [ ]:
!python llm-finetune/generate_lora.py  --prompt "Is pineapple pizza any good?"

### Produce a checkpoint

You can merge your fine-tuned layers on the base model, obtaining a self-contained checkpoint.

In [ ]:
!python llm-finetune/merge_lora.py